In [1]:
import numpy as np
import pandas as pd
import os
import sys
import csv
from sklearn import metrics
from sklearn.metrics import classification_report

sys.path.append("../") 
from bert_sklearn import BertClassifier
from bert_sklearn import load_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
col_names=["label", "sentence"]
#train_amazon = pd.read_csv("amazon_train.csv", names=col_names)
#test_amazon = pd.read_csv("amazon_test.csv")
train_yelp = pd.read_csv("yelp_train.csv", names=col_names)
test_yelp = pd.read_csv("yelp_test.csv", names=col_names)

In [6]:
train_yelp.label.replace(1, 0, inplace=True)
train_yelp.label.replace(2, 1, inplace=True)

test_yelp.label.replace(1, 0, inplace=True)
test_yelp.label.replace(2, 1, inplace=True)

In [7]:
print(test_yelp.head())

   label                                           sentence
0      1  Contrary to other reviews, I have zero complai...
1      0  Last summer I had an appointment to get new ti...
2      1  Friendly staff, same starbucks fair you get an...
3      0  The food is good. Unfortunately the service is...
4      1  Even when we didn't have a car Filene's Baseme...


In [ ]:
%%time
from sklearn.metrics import matthews_corrcoef

X_train = train_yelp['sentence']
y_train = train_yelp['label']

# define model
model = BertClassifier()
model.epochs = 3
model.validation_fraction = 0
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 2

print('\n',model,'\n')

# fit model
model.fit(X_train, y_train)

# test model on dev
X_test = test_yelp['sentence']
y_test = test_yelp['label']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

# Mathews correlation coefficient
print("\nMathews Correlation: %0.2f"%(matthews_corrcoef(y_test, y_pred) * 100))

Building sklearn classifier...

 BertClassifier(bert_model='bert-base-uncased', epochs=3, eval_batch_size=8,
        fp16=False, gradient_accumulation_steps=2, label_list=None,
        learning_rate=2e-05, local_rank=-1, logfile='bert_sklearn.log',
        loss_scale=0, max_seq_length=128, num_mlp_hiddens=500,
        num_mlp_layers=0, random_state=42, restore_file=None,
        train_batch_size=32, use_cuda=True, validation_fraction=0,
        warmup_proportion=0.1) 

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 560000, validation data size: 0


Training:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊    | 34126/35000 [120:36:06<3:14:27, 13.35s/it, loss=0.18]